# Importing Libraries

In [2]:
import numpy as np
import pandas as pd 
import string
import warnings
warnings.filterwarnings('ignore')
import re

#from textblob import TextBlob

import nltk
#nltk.download('punkt')
#nltk.download('wordnet')

import spacy

from nltk.corpus import stopwords

#Stemming
#from nltk.stem import PorterStemmer
#stemmer = PorterStemmer()

#Lemmetization
from nltk.stem import WordNetLemmatizer
lemmetizer = WordNetLemmatizer()

from flask import Flask, redirect, url_for, request,render_template

#Initializing spacy model
nlp = spacy.load('en_core_web_sm')

#Words that we do not require after tokenization in the search term and the title column as well
unwanted_words = ['Email','Sample','Letter','Template','email','sample','letter','template','to','To']

#Punctuations
#punctuations = '''()-[]{};:'"\,<>./?@#$%^&*_`~'''

#Stopwords
stop_words = stopwords.words('english')

# Importing Dataset and Data Preprocessing

In [3]:
#Reading the data set and preprocessing
data = pd.read_excel('templates.xls')
data = data.dropna()
data = data.reset_index(drop=True)

#Replacing 'Letter' with 'Email' || Replacing '-' with ' ' in the cat column
for i in range (0,len(data)):
    data.iloc[i,0] = data.iloc[i,0].replace('Letter','Email')
    data.iloc[i,0] = data.iloc[i,0].replace('letter','Email')
    data.iloc[i,1] = data.iloc[i,1].replace('Letter','Email')
    data.iloc[i,1] = data.iloc[i,1].replace('letter','Email')
    data.iloc[i,0] = data.iloc[i,0].replace('-',' ')

#Adding Search column which will contain tokens of title and arranging the columns in order    
data['Search_column'] = ''
data = data[["Category","Search_column", "Title", "Body"]]

#Removing the 'Download Related Samples...' from footer
for i in range (0,len(data)):
    txt = data.iloc[i,3]
    txt0 = txt.split('\nDownload ')[0]
    data.iloc[i,3] = txt0

#Removing address fields
for i in range (0,len(data)):
    list0 = data.Body[i].split('\n')
    list1 = []
    for string in list0:
        if len(string)> 50:
            list1.append(string)
    body = 'To: [Email id]\n\nSubject: [Subject]\n\n[Salutation]\n\n'
    for j in list1:
        body = body + j + '\n\n'
    body = body + '[Closure]\n\n'+'[Name/Signature]\n'+'[Designation]\n'+'[Organization]\n'
    body = body.replace('Letter','Email')
    body = body.replace('letter','email')
    #print(body)
    data.iloc[i,3] = body

#Preprocessing the title column
for i in range (0,len(data)):
    tex = str(data.Title[i])
    tex_p = ''
    #for char in tex:
    #    if (char not in punctuations):
    #        tex_p = tex_p+char
    tex_p = re.sub(r'[^\w\s]','',tex)
    tex_p = tex_p.strip()
    tex_a = nltk.word_tokenize(tex_p)
    tex_s=[]
    for word in tex_a:
        if (word not in stop_words):
            tex_s.append(word)
    tex_l = []
    for word in tex_s:
        tex_l.append(lemmetizer.lemmatize(word))
    tex_f = []
    for word in tex_l:
        if word not in unwanted_words:
            tex_f.append(word)
        else:
            pass
    data.iloc[i,1] = tex_f

#Named Entity Recognition (Replacing named entities with their tags)
for i in range(0,len(data)):
    s= data.Body[i]
    doc = nlp(s)
    newString = s
    for e in reversed(doc.ents): #reversed to not modify the offsets of other entities when substituting
        start = e.start_char
        end = start + len(e.text)
        newString = newString[:start] + '[' + e.label_ + ']' + newString[end:]
    data.Body[i] = newString
    
data.head()

,Category,Search_column,Title,Body
0,Acceptance Emails,"[Elementary, School, Admission]",Elementary School Admission Email,To: [Email id]\n\nSubject: [Subject]\n\n[Salut...
1,Acceptance Emails,"[Academy, Admission]",Academy Admission Email,To: [Email id]\n\nSubject: [Subject]\n\n[Salut...
2,Acceptance Emails,"[College, Admissions, Recommendation]",College Admissions Recommendation Email,To: [Email id]\n\nSubject: [Subject]\n\n[Salut...
3,Acceptance Emails,"[Decline, Admission]",Decline Admission Email,To: [Email id]\n\nSubject: [Subject]\n\n[Salut...
4,Acceptance Emails,"[Harvard, Acceptance]",Harvard Acceptance Email,To: [Email id]\n\nSubject: [Subject]\n\n[Salut...


# EDA

In [4]:
data.shape

(1574, 4)

Dataset contains 1574 entries and 4 columns

In [5]:
data.columns

Index(['Category', 'Search_column', 'Title', 'Body'], dtype='object')

The dataset consists of 4 columns namely 'Category', 'Search_column', 'Title' and 'Body'.
<br>Dataset altogether consists of 1574 Email templates.

In [6]:
data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1574 entries, 0 to 1573
Data columns (total 4 columns):
 #   Column         Non-Null Count  Dtype 
---  ------         --------------  ----- 
 0   Category       1574 non-null   object
 1   Search_column  1574 non-null   object
 2   Title          1574 non-null   object
 3   Body           1574 non-null   object
dtypes: object(4)
memory usage: 49.3+ KB


As we can see that there are no null values.

In [7]:
print('Length: '+str(len(data.Category.unique()))+'\n')
data.Category.unique()

Length: 53



array(['Acceptance Emails', 'Admission Emails', 'Announcement Emails',
       'Application Emails', 'Appreciation Emails',
       'Authorization Emails', 'Banking Emails', 'Birthday Email',
       'Business Emails', 'Cancellation Email', 'Certification Email',
       'Claim Email', 'Complaint Emails', 'Condolence Emails',
       'Confirmation Email', 'Congratulation Emails', 'Consumer Emails',
       'Customer Emails', 'Decline Email', 'Emails', 'Employment Emails',
       'Finance Emails', 'Formal Email', 'General Emails',
       'Government Emails', 'Grievance Emails', 'Holiday Emails',
       'Inquiry Email', 'Internship Email', 'Interview Email',
       'Introduction Emails', 'Invitation Emails', 'Job Emails',
       'Leave Emails', 'Marketing Emails', 'Order Email',
       'Permission Email', 'Personal Emails', 'Recommendation Emails',
       'Reference Emails', 'Request Emails', 'Resignation Emails',
       'Retirement Emails', 'Sales & Marketing Emails', 'Sales Emails',
       '

As we can see that there are 53 Categories as shown above.

The title column shows the title of each Email

The Search_Column consists of the tokens from the title column for matching the keywords.

The Body column consists of templates of the email.

## Just for future reference

In [8]:
df = pd.DataFrame(data.Category.value_counts())
df = df.reset_index()
df = df.rename(columns={'index':'Category','Category':'N_Letters'})
df.head()

,Category,N_Letters
0,Business Emails,70
1,Personal Emails,70
2,Customer Emails,70
3,General Emails,64
4,Sales & Marketing Emails,60


# Deployment

In [11]:
app = Flask(__name__)

#app.static_folder = 'static'


@app.route('/')
def home_page():
    cats = df.Category.to_list()
    return render_template('index.html',cat_list = cats)



@app.route('/category',methods = ['POST','GET'])
def category():
    cats = df.Category.to_list()
    if request.method == 'POST':
        s = str(request.form['cat'])
        b = data[data.Category == s]
        title = b.Title.to_list()
    return render_template('cat_title.html',title_list=title,cat_list = cats)



@app.route('/category/title',methods = ['POST','GET'])
def cat_title():
    if request.method == 'POST':
        s = str(request.form['title'])
        b = data[data.Title == s]
        b = b.reset_index(drop=True)
        letter = 'Category: '+str(b.Category[0])+'\nTitle: '+str(b.Title[0])+'\n____________________________________________________________________________________________________________\n'+str(b.Body[0])
        letter = letter.split('\n')
        cats = df.Category.to_list()
        c = data[data.Category == b.Category[0]]
        title = c.Title.to_list()
    return render_template('print.html',title_list = title,cat_list = cats,letter=letter)


        
@app.route('/search_results',methods = ['POST','GET'])
def search():
    if request.method == 'POST':
        s = str(request.form['keywords'])
        text=''
        text = re.sub(r'[^\w\s]','',s)
        text = text.strip()
        text = text.title()
        text_a = nltk.word_tokenize(text)
        text_s=[]
        for word in text_a:
            if (word not in stop_words):
                text_s.append(word)
        text_l = []
        for word in text_s:
            if word == 'boss':
                text_l.append(word)
            else: 
                text_l.append(lemmetizer.lemmatize(word))
        text_f = []
        for word in text_l:
            if word not in unwanted_words:
                text_f.append(word)
            else:
                pass
        #Searching for keywords in the search column
        result_in = []
        for word in text_f:
            for i in range (0,len(data)):
                for j in range (0,len(data.iloc[i,1])):
                    if data.iloc[i,1][j] not in text_f:
                        pass
                    else:
                        result_in.append(int(i))
        result_in.sort(reverse=False)
        x = data.iloc[result_in]
        x['priority'] = np.NaN
        p = x.index.value_counts()
        x = x.reset_index()
        p = p.reset_index()
        p = p.rename(columns={0:'priority'},inplace=False)
        for i in range(0,len(x)):
            ind = x['index'][i] 
            for j in range(0,len(p)):
                if p['index'][j] == ind:
                    pr = p['priority'][j]
                else:
                    pass
            x['priority'][i] = pr
        x = x.sort_values(by=['priority'],ascending=False)
        x = x.drop_duplicates(subset=['index'])
        x = x.drop(['index','Search_column','priority'],axis=1)
        x = x.reset_index(drop=True)
        #letter = 'Category: '+str(x.Category[0])+'\nTitle: '+str(x.Title[0])+'\n____________________________________________________________________________________________________________\n'+str(x.Body[0])
        #letter = letter.split('\n')
        cats = df.Category.to_list()
        title_list = x.Title.to_list()
    return render_template('keywordsearch.html',cat_list = cats,title_list=title_list)



@app.route('/search_results/print',methods = ['POST','GET'])
def print_search_results():
    if request.method == 'POST':
        s = str(request.form['title'])
        b = data[data.Title == s]
        b = b.reset_index(drop=True)
        letter = 'Category: '+str(b.Category[0])+'\nTitle: '+str(b.Title[0])+'\n____________________________________________________________________________________________________________\n'+str(b.Body[0])
        letter = letter.split('\n')
        cats = df.Category.to_list()
        c = data[data.Category == b.Category[0]]
        title = c.Title.to_list()
    return render_template('searchresultprint.html',letter = letter,cat_list = cats,title_list=title)




if __name__ == '__main__':
    app.run()

 * Serving Flask app "__main__" (lazy loading)
 * Environment: production
   Use a production WSGI server instead.
 * Debug mode: off


 * Running on http://127.0.0.1:5000/ (Press CTRL+C to quit)
127.0.0.1 - - [29/Sep/2021 14:05:19] "GET / HTTP/1.1" 200 -
127.0.0.1 - - [29/Sep/2021 14:05:36] "POST /category HTTP/1.1" 200 -
127.0.0.1 - - [29/Sep/2021 14:05:42] "POST /category/title HTTP/1.1" 200 -
127.0.0.1 - - [29/Sep/2021 14:05:53] "POST /search_results HTTP/1.1" 200 -
127.0.0.1 - - [29/Sep/2021 14:06:02] "POST /search_results/print HTTP/1.1" 200 -
127.0.0.1 - - [29/Sep/2021 14:08:07] "POST /search_results/print HTTP/1.1" 200 -
127.0.0.1 - - [29/Sep/2021 14:08:24] "POST /category HTTP/1.1" 200 -
127.0.0.1 - - [29/Sep/2021 14:08:42] "POST /category HTTP/1.1" 200 -
127.0.0.1 - - [29/Sep/2021 14:08:52] "POST /category/title HTTP/1.1" 200 -


# Final Combined Deployment

In [10]:
#****Importing Libraries****#

import numpy as np
import pandas as pd 
import string
import warnings
warnings.filterwarnings('ignore')
import re

#from textblob import TextBlob

import nltk
#nltk.download('punkt')
#nltk.download('wordnet')

import spacy

from nltk.corpus import stopwords

#Stemming
#from nltk.stem import PorterStemmer
#stemmer = PorterStemmer()

#Lemmetization
from nltk.stem import WordNetLemmatizer
lemmetizer = WordNetLemmatizer()

from flask import Flask, redirect, url_for, request,render_template

#Initializing spacy model
nlp = spacy.load('en_core_web_sm')

#Words that we do not require after tokenization in the search term and the title column as well
unwanted_words = ['Email','Sample','Letter','Template','email','sample','letter','template','to','To']

#Punctuations
#punctuations = '''()-[]{};:'"\,<>./?@#$%^&*_`~'''

#Stopwords
stop_words = stopwords.words('english')


#****Importing Dataset and data Preprocessing****#

#Reading the data set and preprocessing
data = pd.read_excel('templates.xls')
data = data.dropna()
data = data.reset_index(drop=True)

#Replacing 'Letter' with 'Email' || Replacing '-' with ' ' in the cat column
for i in range (0,len(data)):
    data.iloc[i,0] = data.iloc[i,0].replace('Letter','Email')
    data.iloc[i,0] = data.iloc[i,0].replace('letter','Email')
    data.iloc[i,1] = data.iloc[i,1].replace('Letter','Email')
    data.iloc[i,1] = data.iloc[i,1].replace('letter','Email')
    data.iloc[i,0] = data.iloc[i,0].replace('-',' ')

#Adding Search column which will contain tokens of title and arranging the columns in order    
data['Search_column'] = ''
data = data[["Category","Search_column", "Title", "Body"]]

#Removing the 'Download Related Samples...' from footer
for i in range (0,len(data)):
    txt = data.iloc[i,3]
    txt0 = txt.split('\nDownload ')[0]
    data.iloc[i,3] = txt0

#Removing address fields
for i in range (0,len(data)):
    list0 = data.Body[i].split('\n')
    list1 = []
    for string in list0:
        if len(string)> 50:
            list1.append(string)
    body = 'To: [Email id]\n\nSubject: [Subject]\n\n[Salutation]\n\n'
    for j in list1:
        body = body + j + '\n\n'
    body = body + '[Closure]\n\n'+'[Name/Signature]\n'+'[Designation]\n'+'[Organization]\n'
    body = body.replace('Letter','Email')
    body = body.replace('letter','email')
    #print(body)
    data.iloc[i,3] = body

#Preprocessing the title column
for i in range (0,len(data)):
    tex = str(data.Title[i])
    tex_p = ''
    #for char in tex:
    #    if (char not in punctuations):
    #        tex_p = tex_p+char
    tex_p = re.sub(r'[^\w\s]','',tex)
    tex_p = tex_p.strip()
    tex_a = nltk.word_tokenize(tex_p)
    tex_s=[]
    for word in tex_a:
        if (word not in stop_words):
            tex_s.append(word)
    tex_l = []
    for word in tex_s:
        tex_l.append(lemmetizer.lemmatize(word))
    tex_f = []
    for word in tex_l:
        if word not in unwanted_words:
            tex_f.append(word)
        else:
            pass
    data.iloc[i,1] = tex_f

#Named Entity Recognition (Replacing named entities with their tags)
for i in range(0,len(data)):
    s= data.Body[i]
    doc = nlp(s)
    newString = s
    for e in reversed(doc.ents): #reversed to not modify the offsets of other entities when substituting
        start = e.start_char
        end = start + len(e.text)
        newString = newString[:start] + '[' + e.label_ + ']' + newString[end:]
    data.Body[i] = newString
    

#****For future reference****#

df = pd.DataFrame(data.Category.value_counts())
df = df.reset_index()
df = df.rename(columns={'index':'Category','Category':'N_Letters'})


#****Deployment****#

app = Flask(__name__)

#app.static_folder = 'static'


@app.route('/')
def home_page():
    cats = df.Category.to_list()
    return render_template('index.html',cat_list = cats)



@app.route('/category',methods = ['POST','GET'])
def category():
    cats = df.Category.to_list()
    if request.method == 'POST':
        s = str(request.form['cat'])
        b = data[data.Category == s]
        title = b.Title.to_list()
    return render_template('cat_title.html',title_list=title,cat_list = cats)



@app.route('/category/title',methods = ['POST','GET'])
def cat_title():
    if request.method == 'POST':
        s = str(request.form['title'])
        b = data[data.Title == s]
        b = b.reset_index(drop=True)
        letter = 'Category: '+str(b.Category[0])+'\nTitle: '+str(b.Title[0])+'\n____________________________________________________________________________________________________________\n'+str(b.Body[0])
        letter = letter.split('\n')
        cats = df.Category.to_list()
        c = data[data.Category == b.Category[0]]
        title = c.Title.to_list()
    return render_template('print.html',title_list = title,cat_list = cats,letter=letter)


        
@app.route('/search_results',methods = ['POST','GET'])
def search():
    if request.method == 'POST':
        s = str(request.form['keywords'])
        text=''
        text = re.sub(r'[^\w\s]','',s)
        text = text.strip()
        text = text.title()
        text_a = nltk.word_tokenize(text)
        text_s=[]
        for word in text_a:
            if (word not in stop_words):
                text_s.append(word)
        text_l = []
        for word in text_s:
            if word == 'boss':
                text_l.append(word)
            else: 
                text_l.append(lemmetizer.lemmatize(word))
        text_f = []
        for word in text_l:
            if word not in unwanted_words:
                text_f.append(word)
            else:
                pass
        #Searching for keywords in the search column
        result_in = []
        for word in text_f:
            for i in range (0,len(data)):
                for j in range (0,len(data.iloc[i,1])):
                    if data.iloc[i,1][j] not in text_f:
                        pass
                    else:
                        result_in.append(int(i))
        result_in.sort(reverse=False)
        x = data.iloc[result_in]
        x['priority'] = np.NaN
        p = x.index.value_counts()
        x = x.reset_index()
        p = p.reset_index()
        p = p.rename(columns={0:'priority'},inplace=False)
        for i in range(0,len(x)):
            ind = x['index'][i] 
            for j in range(0,len(p)):
                if p['index'][j] == ind:
                    pr = p['priority'][j]
                else:
                    pass
            x['priority'][i] = pr
        x = x.sort_values(by=['priority'],ascending=False)
        x = x.drop_duplicates(subset=['index'])
        x = x.drop(['index','Search_column','priority'],axis=1)
        x = x.reset_index(drop=True)
        #letter = 'Category: '+str(x.Category[0])+'\nTitle: '+str(x.Title[0])+'\n____________________________________________________________________________________________________________\n'+str(x.Body[0])
        #letter = letter.split('\n')
        cats = df.Category.to_list()
        title_list = x.Title.to_list()
    return render_template('keywordsearch.html',cat_list = cats,title_list=title_list)



@app.route('/search_results/print',methods = ['POST','GET'])
def print_search_results():
    if request.method == 'POST':
        s = str(request.form['title'])
        b = data[data.Title == s]
        b = b.reset_index(drop=True)
        letter = 'Category: '+str(b.Category[0])+'\nTitle: '+str(b.Title[0])+'\n____________________________________________________________________________________________________________\n'+str(b.Body[0])
        letter = letter.split('\n')
        cats = df.Category.to_list()
        c = data[data.Category == b.Category[0]]
        title = c.Title.to_list()
    return render_template('searchresultprint.html',letter = letter,cat_list = cats,title_list=title)




if __name__ == '__main__':
    app.run()

 * Serving Flask app "__main__" (lazy loading)
 * Environment: production
   Use a production WSGI server instead.
 * Debug mode: off


 * Running on http://127.0.0.1:5000/ (Press CTRL+C to quit)
